In [17]:
import cv2
import cvlib as cv
import sys
import numpy as np
from numpy.lib.type_check import imag
import matplotlib.pyplot as plt

In [23]:
ls

 Volume in drive C is Windows
 Volume Serial Number is 088F-1ED4

 Directory of C:\Users\EREN\Documents\YapayZeka\GitHub\21Project\Gender Dedection

12/24/2023  07:39 PM    <DIR>          .
12/24/2023  07:39 PM    <DIR>          ..
12/24/2023  07:19 PM    <DIR>          .ipynb_checkpoints
08/29/2019  11:34 PM             2,481 age_deploy.prototxt
08/29/2019  11:29 PM        45,661,480 age_net.caffemodel
12/24/2023  07:30 PM        86,389,424 drive-download-20231224T162849Z-001.zip
12/24/2023  07:39 PM             9,484 Gender Dedection.ipynb
08/29/2019  11:35 PM             2,482 gender_deploy.prototxt
08/29/2019  11:31 PM        45,649,168 gender_net.caffemodel
08/29/2019  11:37 PM            37,272 opencv_face_detector.pbtxt
08/29/2019  11:35 PM         2,727,750 opencv_face_detector_uint8.pb
               8 File(s)    180,479,541 bytes
               3 Dir(s)  63,327,686,656 bytes free


In [4]:
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("Cannot open camera")
    exit()

In [24]:
# Load the pre-trained face detection model
face_prototxt_path = 'opencv_face_detection_model.prototxt'
face_caffemodel_path = 'opencv_face_detection_model.caffemodel'
face_net = cv2.dnn.readNetFromCaffe(face_prototxt_path, face_caffemodel_path)

# Load the pre-trained age and gender prediction model
age_gender_prototxt_path = 'age_gender_model.prototxt'
age_gender_caffemodel_path = 'age_gender_model.caffemodel'
age_gender_net = cv2.dnn.readNetFromCaffe(age_gender_prototxt_path, age_gender_caffemodel_path)


error: OpenCV(4.8.1) D:\a\opencv-python\opencv-python\opencv\modules\dnn\src\caffe\caffe_io.cpp:1126: error: (-2:Unspecified error) FAILED: fs.is_open(). Can't open "opencv_face_detection_model.prototxt" in function 'cv::dnn::ReadProtoFromTextFile'


In [14]:

# Categories of distribution 
la = ['(0-2)', '(4-6)', '(8-12)', '(15-20)', 
      '(25-32)', '(38-43)', '(48-53)', '(60-100)'] 
lg = ['Male', 'Female'] 

In [21]:
while True:
    # Capture frame-by-frame
    ret, frame = cap.read()
    
    # if frame is read correctly ret is True
    if not ret:
        print("Can't receive frame (stream end?). Exiting ...")
        break

    
    # Copy image 
    fr_cv = frame.copy()

    # Face detection 
    fr_h = fr_cv.shape[0] 
    fr_w = fr_cv.shape[1] 
    blob = cv2.dnn.blobFromImage(fr_cv, 1.0, (300, 300), 
                                 [104, 117, 123], True, False) 

    #face.setInput(blob) 
    detections = face.forward()
    
    
    
    # Face bounding box creation 
    faceBoxes = [] 
    for i in range(detections.shape[2]): 

        #Bounding box creation if confidence > 0.7 
        confidence = detections[0, 0, i, 2] 
        if confidence > 0.7: 

            x1 = int(detections[0, 0, i, 3]*fr_w) 
            y1 = int(detections[0, 0, i, 4]*fr_h) 
            x2 = int(detections[0, 0, i, 5]*fr_w) 
            y2 = int(detections[0, 0, i, 6]*fr_h) 

            faceBoxes.append([x1, y1, x2, y2]) 

            cv2.rectangle(fr_cv, (x1, y1), (x2, y2), 
                          (0, 255, 0), int(round(fr_h/150)), 8) 

    # Checking if face detected or not 
    if not faceBoxes: 
        print("No face detected") 

    # Final results (otherwise) 
    # Loop for all the faces detected 
    for faceBox in faceBoxes: 

        #Extracting face as per the faceBox 
        face = fr_cv[max(0, faceBox[1]-15): 
                     min(faceBox[3]+15, fr_cv.shape[0]-1), 
                     max(0, faceBox[0]-15):min(faceBox[2]+15, 
                                   fr_cv.shape[1]-1)] 

        #Extracting the main blob part 
        blob = cv2.dnn.blobFromImage( 
            face, 1.0, (227, 227), MODEL_MEAN_VALUES, swapRB=False) 

        #Prediction of gender 
        #gen.setInput(blob) 
        genderPreds = gen.forward() 
        gender = lg[genderPreds[0].argmax()] 

        #Prediction of age 
        age.setInput(blob) 
        agePreds = age.forward() 
        age = la[agePreds[0].argmax()] 

        #Putting text of age and gender  
        #At the top of box 
        cv2.putText(fr_cv, 
                    f'{gender}, {age}', 
                    (faceBox[0]-150, faceBox[1]+10), 
                    cv2.FONT_HERSHEY_SIMPLEX, 
                    1.3, 
                    (217, 0, 0), 
                    4, 
                    cv2.LINE_AA) 

        plt.figure(figsize=(7, 7)) 
        plt.imshow(fr_cv)
    
    # Görüntüleme
    cv2.imshow("Gender Detection", frame)
    if cv.waitKey(1) == ord('q'):
        break
# Kamera İşlemlerini Bitir
cap.release() #Kamerayı Kapat
cv.destroyAllWindows() #Formu Kapat

AttributeError: 'numpy.ndarray' object has no attribute 'forward'